# Train Graph NN on Call Mentions

In [1]:
# Settings
model_name = 'exclude_previous_rating_model_2'
inductive_or_transductive = 'Inductive'

In [2]:
# Packages
from Graph_NN_Functions import *

## Load Learnable Network Data

In [3]:
# Load feature and class data
feature_and_class_df = pd.read_excel('../../../Data/Learnable Network/feature_and_class_df.xlsx')
feature_and_class_df

,ticker,fixed_quarter_date,earnings_call_date,Rating,rating_date,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,next_rating_date_or_end_of_data,...,Ovrst,Undrst,PN,SW,AP,OU,tone,num_q_by_len,train_test_80_20,node
0,AAPL,2014-10-01,2014-07-22,AA,2014-05-27,AA,2015-02-18,AAA,2014-04-24,2015-02-18,...,364,131,5.518519,15.261905,2.661290,2.778626,3.188264,0.003822,train,1208
1,AAPL,2015-01-01,2014-10-20,AA,2014-05-27,AA,2015-02-18,AAA,2014-04-24,2015-02-18,...,465,152,5.348485,15.934783,3.296482,3.059211,3.681858,0.002766,train,334
2,AAPL,2015-04-01,2015-01-27,AA,2015-02-18,AA,2015-05-28,AA,2014-05-27,2015-05-28,...,468,151,3.927711,8.113636,2.841346,3.099338,1.307366,0.004628,train,1694
3,AAPL,2015-07-01,2015-04-27,AA,2015-06-02,AA,2015-08-25,AA,2015-05-28,2015-08-25,...,415,135,5.250000,9.142857,2.640187,3.074074,2.025933,0.003861,train,90
4,AAPL,2015-10-01,2015-07-21,AA,2015-08-25,AA,2016-05-20,AA,2015-06-02,2016-05-20,...,449,148,4.209877,10.442857,2.579909,3.033784,1.815531,0.003915,train,808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1958,ZBRA,2016-01-01,2015-11-10,B,2015-09-03,B,2016-09-26,NaN,NaN,2016-09-26,...,431,202,2.662338,13.586667,2.768293,2.133663,0.982796,0.005321,train,141
1959,ZBRA,2016-04-01,2016-02-25,B,2015-09-03,B,2016-09-26,NaN,NaN,2016-09-26,...,449,173,3.991071,13.574713,2.382550,2.595376,1.925784,0.002704,train,40
1960,ZBRA,2016-07-01,2016-05-10,B,2015-09-03,B,2016-09-26,NaN,NaN,2016-09-26,...,394,197,2.671053,8.080357,2.446970,2.000000,-0.423271,0.002775,train,1449
1961,ZBRA,2016-10-01,2016-08-09,B,2016-09-26,NaN,NaN,B,2015-09-03,2016-12-31,...,403,183,3.625954,12.151163,3.066116,2.202186,1.210391,0.003215,test,298


In [4]:
src_dst_df = pd.read_excel('../../../Data/Learnable Network/src_dst_df.xlsx')
src_dst_df

,src,dst
0,119,1525
1,870,1525
2,1359,1525
3,1907,1525
4,491,1525
...,...,...
1800,1124,1454
1801,619,434
1802,1279,996
1803,14,484


In [5]:
node_to_int = pd.read_excel('../../../Data/Learnable Network/node_to_int.xlsx')
# convert to dictionary
node_to_int = node_to_int.set_index('node').to_dict()['node_int']
node_to_int

{'PCAR : 2011-10-01': 0,
 'HOLX : 2014-07-01': 1,
 'PTC : 2016-10-01': 2,
 'INTC : 2016-07-01': 3,
 'IBM : 2013-07-01': 4,
 'MTSI : 2016-07-01': 5,
 'SRE : 2014-01-01': 6,
 'PCAR : 2014-10-01': 7,
 'HOLX : 2015-01-01': 8,
 'UIS : 2012-04-01': 9,
 'SBAC : 2015-01-01': 10,
 'SCHL : 2016-04-01': 11,
 'AMGN : 2016-04-01': 12,
 'LRCX : 2015-01-01': 13,
 'HOV : 2015-10-01': 14,
 'APD : 2016-01-01': 15,
 'MERC : 2016-04-01': 16,
 'MCHP : 2015-04-01': 17,
 'MTDR : 2016-07-01': 18,
 'WWW : 2016-07-01': 19,
 'KMI : 2016-01-01': 20,
 'RS : 2012-10-01': 21,
 'XOM : 2016-07-01': 22,
 'NCR : 2013-10-01': 23,
 'KOS : 2016-10-01': 24,
 'ADSK : 2016-10-01': 25,
 'NFLX : 2016-01-01': 26,
 'TEX : 2013-01-01': 27,
 'MSI : 2013-04-01': 28,
 'SANM : 2016-01-01': 29,
 'MSI : 2014-04-01': 30,
 'MGM : 2016-10-01': 31,
 'WM : 2016-10-01': 32,
 'FLR : 2012-07-01': 33,
 'SSP : 2016-01-01': 34,
 'EQIX : 2015-07-01': 35,
 'NWE : 2014-01-01': 36,
 'EXC : 2014-10-01': 37,
 'FLEX : 2014-07-01': 38,
 'ATVI : 2016-07-01

In [6]:
# Load column names
numeric_feature_columns, cat_feature_columns, target_column, custom_mapping = get_column_names_and_mapping(model_name)

In [7]:
# Prepare matrices
X_train_scaled, X_test_scaled, y_train, y_test, feature_names, train_ticker_by_fixed_quarter_date, test_ticker_by_fixed_quarter_date = prepare_matrices(feature_and_class_df, numeric_feature_columns, cat_feature_columns, target_column, custom_mapping)

feature names: 
['num__EBIT' 'num__common_plus_preferred_stock' 'num__workingCapital'
 'num__Ratio_A' 'num__Ratio_B' 'num__Ratio_C' 'num__Ratio_D'
 'num__Ratio_E' 'num__cashAndCashEquivalents' 'num__shortTermInvestments'
 'num__cashAndShortTermInvestments' 'num__netReceivables'
 'num__inventory_balance_sheet' 'num__otherCurrentAssets'
 'num__totalCurrentAssets' 'num__propertyPlantEquipmentNet'
 'num__goodwill' 'num__intangibleAssets'
 'num__goodwillAndIntangibleAssets' 'num__longTermInvestments'
 'num__taxAssets' 'num__otherNonCurrentAssets'
 'num__totalNonCurrentAssets' 'num__otherAssets' 'num__totalAssets'
 'num__accountPayables' 'num__shortTermDebt' 'num__taxPayables'
 'num__deferredRevenue' 'num__otherCurrentLiabilities'
 'num__totalCurrentLiabilities' 'num__longTermDebt'
 'num__deferredRevenueNonCurrent' 'num__deferredTaxLiabilitiesNonCurrent'
 'num__otherNonCurrentLiabilities' 'num__totalNonCurrentLiabilities'
 'num__otherLiabilities' 'num__capitalLeaseObligations'
 'num__totalLi

In [8]:
# Assemble back into dataframes

# Train and val
train_and_val_df = pd.DataFrame(X_train_scaled, columns=feature_names)
print(train_and_val_df)
# Add y_train
train_and_val_df[target_column] = y_train.reset_index(drop=True)
# Add ticker by fixed quarter date
train_and_val_df = pd.concat([train_ticker_by_fixed_quarter_date.reset_index(drop=True).sort_values(['ticker', 'fixed_quarter_date']), train_and_val_df], axis=1)
# Add node by merging with feature_and_class_df (inner join)
train_and_val_df = train_and_val_df.merge(feature_and_class_df[['ticker', 'fixed_quarter_date', 'node']], on=['ticker', 'fixed_quarter_date'], how='inner')
# Drop ticker and fixed_quarter_date
train_and_val_df = train_and_val_df.drop(['ticker', 'fixed_quarter_date'], axis=1)

# Test
test_df = pd.DataFrame(X_test_scaled, columns=feature_names)
# Add y_test
test_df[target_column] = y_test.reset_index(drop=True)
# Add ticker by fixed quarter date
test_df = pd.concat([test_ticker_by_fixed_quarter_date.reset_index(drop=True).sort_values(['ticker', 'fixed_quarter_date']), test_df], axis=1)
# Add node by merging with feature_and_class_df (inner join)
test_df = test_df.merge(feature_and_class_df[['ticker', 'fixed_quarter_date', 'node']], on=['ticker', 'fixed_quarter_date'], how='inner')
# Drop ticker and fixed_quarter_date
test_df = test_df.drop(['ticker', 'fixed_quarter_date'], axis=1)

print('finalized dfs')
print(train_and_val_df)
print(test_df)
print('missing values of target_column in train_and_val_df or test_df?')
print(train_and_val_df[target_column].isnull().sum() > 0)
print(test_df[target_column].isnull().sum() > 0)

      num__EBIT  num__common_plus_preferred_stock  num__workingCapital  \
0     -0.550946                         -0.405512            -0.220795   
1     -0.549646                         -0.404515            -0.224155   
2     -0.530384                         -0.403773            -0.223205   
3     -0.539172                         -0.402764            -0.223332   
4     -0.545349                         -0.401957            -0.224037   
...         ...                               ...                  ...   
1557   0.081791                          1.214090            -0.507520   
1558  -0.466756                         -0.423693            -0.106127   
1559  -0.454378                         -0.423693            -0.133219   
1560  -0.502770                         -0.423457            -0.142898   
1561  -0.271596                         -0.420061             0.254596   

      num__Ratio_A  num__Ratio_B  num__Ratio_C  num__Ratio_D  num__Ratio_E  \
0         1.365738     -0.223037 

## Run the Model

In [9]:
run_model(train_and_val_df = train_and_val_df,
         test_df = test_df,
         src_dst_df = src_dst_df,
         model_dir = '../../../Output/Modelling/Graph Neural Network/' + inductive_or_transductive + '/' + model_name + '/',
         prediction_file_path = '../../../Data/Predictions/Graph Neural Network/' + inductive_or_transductive + '/' + model_name + '_predictions.xlsx',
         target_column = target_column,
         custom_mapping = custom_mapping,
         node_to_int = node_to_int,
         n_hidden = 128,
         n_layers = 2,
         dropout = 0.4,
         weight_decay = 1e-2,
         n_epochs = 150,
         lr = 0.01,
         aggregator_type = "pool",
         inductive = (inductive_or_transductive == 'Inductive'))

Further slice the train dataset into train and validation datasets.
The training data has shape: (1249, 141).
The validation data has shape: (313, 141).
The test data has shape: (401, 141).
Generate train, validation, and test masks.
sum of train mask
tensor(1249)
sum of val mask
tensor(313)
sum of test mask
tensor(401)
Number of nodes = 1963
Number of features for each node = 139
Number of classes = 8.
Initializing Model
Initialized Model
NodeClassification(
  (gconv_model): GraphSAGEModel(
    (layers): ModuleList(
      (0): SAGEConv(
        (feat_drop): Dropout(p=0.4, inplace=False)
        (fc_pool): Linear(in_features=139, out_features=139, bias=True)
        (fc_neigh): Linear(in_features=139, out_features=128, bias=False)
        (fc_self): Linear(in_features=139, out_features=128, bias=True)
      )
      (1): SAGEConv(
        (feat_drop): Dropout(p=0.4, inplace=False)
        (fc_pool): Linear(in_features=128, out_features=128, bias=True)
        (fc_neigh): Linear(in_featu